In [19]:
import {InferenceSession, Tensor} from 'onnxruntime-web';
import * as ort from "onnxruntime-common";
import * as nj from 'numjs'
// import {convertArrayToImg} from '../src/app/helpers/imageHelpers'

let tensor = new ort.Tensor(
    new Float32Array(256 * 256).fill(1),
    [1, 1, 256, 256]
  );
const imageUint8 = nj.array(nj.uint8(new Float32Array(25 * 25).fill(1)));
// const reshape = nj.ones([2,1], 'uint8')

console.log(imageUint8.reshape(5,125))
console.log(reshape.shape, reshape)
let img = nj.images.data.digit;
const resizedImage = nj.images.resize(img, 1024, 1024);

console.log(resizedImage)


11:26 - Type '2' is not assignable to type '1'.


In [66]:
function nearestNeighbor(src, dst) {
    const wSrc = src.shape[0];
    const hSrc = src.shape[1];

    const wDst = dst.shape[0];
    const hDst = dst.shape[1];

    const bufSrc = src.flatten().tolist();
    const bufDst = dst.flatten().tolist();
    console.log("test1 ", wSrc, hSrc, wDst, hDst)
    for (let i = 0; i < hDst; i++) {
      for (let j = 0; j < wDst; j++) {
        let posDst = (i * wDst + j) ;

        const iSrc = Math.floor((i * hSrc) / hDst);
        const jSrc = Math.floor((j * wSrc) / wDst);
        let posSrc = (iSrc * wSrc + jSrc) ;
        // console.log("test ", iSrc, jSrc, posSrc, posDst)
        bufDst[posDst++] = bufSrc[posSrc++];
        // bufDst[posDst++] = bufSrc[posSrc++];
        // bufDst[posDst++] = bufSrc[posSrc++];
        // bufDst[posDst++] = bufSrc[posSrc++];
      }
    }
    console.log(bufDst, bufSrc)
};

In [79]:
function interpolate2D(src, dst, interpolate) {
    const bufSrc = src.flatten().tolist();
    const bufDst = dst.flatten().tolist();

    const wSrc = src.shape[0];
    const hSrc = src.shape[1];

    const wDst = dst.shape[0];
    const hDst = dst.shape[1];

    // when dst smaller than src/2, interpolate first to a multiple between 0.5 and 1.0 src, then sum squares
    const wM = Math.max(1, Math.floor(wSrc / wDst));
    const wDst2 = wDst * wM;
    const hM = Math.max(1, Math.floor(hSrc / hDst));
    const hDst2 = hDst * hM;

    // ===========================================================
    // Pass 1 - interpolate rows
    // buf1 has width of dst2 and height of src
    const buf1 = Buffer.alloc(wDst2 * hSrc);
    for (let i = 0; i < hSrc; i++) {
      for (let j = 0; j < wDst2; j++) {
        // i in src coords, j in dst coords

        // calculate x in src coords
        // this interpolation requires 4 sample points and the two inner ones must be real
        // the outer points can be fudged for the edges.
        // therefore (wSrc-1)/wDst2
        const x = (j * (wSrc - 1)) / wDst2;
        const xPos = Math.floor(x);
        const t = x - xPos;
        const srcPos = (i * wSrc + xPos);
        const buf1Pos = (i * wDst2 + j);

        for (let k = 0; k < 1; k++) {
          const kPos = srcPos + k;
          const x0 =
            xPos > 0 ? bufSrc[kPos - 4] : 2 * bufSrc[kPos] - bufSrc[kPos + 4];
          const x1 = bufSrc[kPos];
          const x2 = bufSrc[kPos + 4];
          const x3 =
            xPos < wSrc - 2
              ? bufSrc[kPos + 8]
              : 2 * bufSrc[kPos + 4] - bufSrc[kPos];
          buf1[buf1Pos + k] = interpolate(x0, x1, x2, x3, t);
        }
      }
    }
    // this._writeFile(wDst2, hSrc, buf1, "out/buf1.jpg");

    // ===========================================================
    // Pass 2 - interpolate columns
    // buf2 has width and height of dst2
    const buf2 = Buffer.alloc(wDst2 * hDst2);
    for (let i = 0; i < hDst2; i++) {
      for (let j = 0; j < wDst2; j++) {
        // i&j in dst2 coords

        // calculate y in buf1 coords
        // this interpolation requires 4 sample points and the two inner ones must be real
        // the outer points can be fudged for the edges.
        // therefore (hSrc-1)/hDst2
        const y = (i * (hSrc - 1)) / hDst2;
        const yPos = Math.floor(y);
        const t = y - yPos;
        const buf1Pos = (yPos * wDst2 + j);
        const buf2Pos = (i * wDst2 + j);
        for (let k = 0; k < 1; k++) {
          const kPos = buf1Pos + k;
          const y0 =
            yPos > 0
              ? buf1[kPos - wDst2]
              : 2 * buf1[kPos] - buf1[kPos + wDst2];
          const y1 = buf1[kPos];
          const y2 = buf1[kPos + wDst2];
          const y3 =
            yPos < hSrc - 2
              ? buf1[kPos + wDst2 * 8]
              : 2 * buf1[kPos + wDst2] - buf1[kPos];

          buf2[buf2Pos + k] = interpolate(y0, y1, y2, y3, t);
        }
      }
    }
    // this._writeFile(wDst2, hDst2, buf2, "out/buf2.jpg");

    // ===========================================================
    // Pass 3 - scale to dst
    const m = wM * hM;
    if (m > 1) {
      for (let i = 0; i < hDst; i++) {
        for (let j = 0; j < wDst; j++) {
          // i&j in dst bounded coords
          let r = 0;
          let g = 0;
          let b = 0;
          let a = 0;
          let realColors = 0;

          for (let y = 0; y < hM; y++) {
            const yPos = i * hM + y;

            for (let x = 0; x < wM; x++) {
              const xPos = j * wM + x;
              const xyPos = (yPos * wDst2 + xPos);
              const pixelAlpha = buf2[xyPos + 3];

              if (pixelAlpha) {
                r += buf2[xyPos];
                g += buf2[xyPos + 1];
                b += buf2[xyPos + 2];
                realColors++;
              }

              a += pixelAlpha;
            }
          }

          const pos = (i * wDst + j);
          bufDst[pos] = realColors ? Math.round(r / realColors) : 0;
          // bufDst[pos + 1] = realColors ? Math.round(g / realColors) : 0;
          // bufDst[pos + 2] = realColors ? Math.round(b / realColors) : 0;
          // bufDst[pos + 3] = Math.round(a / m);
        }
      }
      console.log(bufDst)
    } else {
      // replace dst buffer with buf2
      dst = buf2;
    }
  }

function bicubicInterpolation(src, dst) {
    const interpolateCubic = function (x0, x1, x2, x3, t) {
      const a0 = x3 - x2 - x0 + x1;
      const a1 = x0 - x1 - a0;
      const a2 = x2 - x0;
      const a3 = x1;
      return Math.max(
        0,
        Math.min(255, a0 * (t * t * t) + a1 * (t * t) + a2 * t + a3)
      );
    };

    return interpolate2D(src, dst, interpolateCubic);
  }

In [80]:
const imageUint8 = nj.images.data.digit
const image = nj.array(nj.uint8(new Float32Array(5 * 5).fill(0))).reshape(5,5);

bicubicInterpolation(imageUint8, image)


[
  0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0,
  0
]


In [1]:
const float32Data = new Float32Array(11*22).fill(1);

In [3]:
const matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
];
// Print the entire matrix to the console
console.log(matrix);
console.log(matrix[0]);


[ [ 1, 2, 3 ], [ 4, 5, 6 ], [ 7, 8, 9 ] ]
[ 1, 2, 3 ]


In [19]:
import {InferenceSession, Tensor, env} from 'onnxruntime-web';
const fs = require('fs')


// use an async context to call onnxruntime functions.
async function main() {
    try {

        // const float32Data = new Float32Array(160*256*256*1).fill(1);
        // const inputTensor = new Tensor("float32", float32Data, [160,256,256,1]);
        env.debug = true;
        env.logLevel = 'verbose';
        const float32Data = new Float32Array(211*224*224).fill(1);
        const inputTensor = new Tensor("float32", float32Data, [211,224,224,1]);
        // inputTensor.reshape([211,224,224,1])
        console.log(inputTensor.data)
        // data = imageDataToTensor(nvimage.img, [211,224,224,1])
        

        var session = await InferenceSession.create('../public/model/model.onnx');
        console.log(session.inputNames[0])

        // prepare feeds. use model input names as keys.
        var feeds = { input_1:inputTensor };


        // feed inputs and run
        var results = await session.run(feeds, ['conv2d_transpose_4']);

        // read from results
        const dataC = results;
        console.log(`data of result tensor 'c': ${dataC}`);

        return dataC

    } catch (e) {
        console.error(`failed to inference ONNX model: ${e}.`);
    }
}

var res = await main();
console.log(res)

Float32Array(10587136) [
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1,
  ... 10587036 more items
]
input_1
data of result tensor 'c': [object Object]
{
  conv2d_transpose_4: l {
    dims: [ 211, 224, 224, 1 ],
    type: 'float32',
    data: Float32Array(10587136) [
      0.002875823061913252,
      -0.0035482447128742933,
      0.0015134376008063555,
      -0.00024906962062232196,
      -0.0005567597690969706,
      -0.0010869965190067887,
      0.0001240931887878105,
      0.00011437403009040281,
      -0.00022227723093237728,
      0.0006404236191883683,
      0.00023755755682941526,
      0.00027328336727805436,
      0.0003408203483559191,
      0.0003292314358986914,
      -0.000867693976033479,
      -0.000025216

In [15]:
import {InferenceSession, Tensor} from 'onnxruntime-node';
const fs = require('fs')


// use an async context to call onnxruntime functions.
async function main() {
    try {

        // const float32Data = new Float32Array(160*256*256*1).fill(1);
        // const inputTensor = new Tensor("float32", float32Data, [160,256,256,1]);
        const float32Data = new Float32Array(211*224*224).fill(1);
        const inputTensor = new Tensor("float32", float32Data, [211,224,224,1]);
        // inputTensor.reshape([211,224,224,1])
        console.log(inputTensor.data)
        // data = imageDataToTensor(nvimage.img, [211,224,224,1])
        

        var session = await InferenceSession.create('../public/model/model.onnx');
        console.log(session.inputNames[0])

        // prepare feeds. use model input names as keys.
        var feeds = { input_1:inputTensor };


        // feed inputs and run
        var results = await session.run(feeds, ['conv2d_transpose_4']);

        // read from results
        const dataC = results;
        console.log(`data of result tensor 'c': ${dataC}`);

        return dataC

    } catch (e) {
        console.error(`failed to inference ONNX model: ${e}.`);
    }
}

var res = await main();
console.log(res)

Float32Array(10587136) [
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1,
  ... 10587036 more items
]
input_1
data of result tensor 'c': [object Object]
{
  conv2d_transpose_4: l {
    dims: [ 211, 224, 224, 1 ],
    type: 'float32',
    data: Float32Array(10587136) [
      0.002875823061913252,
      -0.0035482447128742933,
      0.0015134376008063555,
      -0.00024906962062232196,
      -0.0005567597690969706,
      -0.0010869965190067887,
      0.0001240931887878105,
      0.00011437403009040281,
      -0.00022227723093237728,
      0.0006404236191883683,
      0.00023755755682941526,
      0.00027328336727805436,
      0.0003408203483559191,
      0.0003292314358986914,
      -0.000867693976033479,
      -0.000025216

In [7]:
function imageDataToTensor(data, dims){
  // 1a. Extract the R, G, and B channels from the data to form a 3D int array
  const [R, G, B] = new Array([], [], []);
  for (let i = 0; i < data.length; i += 4) {
    R.push(data[i]);
    G.push(data[i + 1]);
    B.push(data[i + 2]);
    // 2. skip data[i + 3] thus filtering out the alpha channel
  }
  ///console.log(R);
  //console.log(G);
  //console.log(B);
  // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
  const transposedData = R.concat(G).concat(B);

  // 3. convert to float32
  let i, l = transposedData.length; // length, we need this for the loop
  const float32Data = new Float32Array(dims[0]*dims[1]*dims[2]); // create the Float32Array for output
  for (i = 0; i < l; i++) {
    float32Data[i] = transposedData[i] / 255.0; // convert to float
  }

  const inputTensor = new Tensor("float32", float32Data, dims);
  return inputTensor;
}

In [20]:
let float32Data = new Float32Array(160*256*256*1).fill(0);
console.log(float32Data.length)
const newArray = Array.from({...float32Data, length:10587136}).fill(2)
float32Data.splice(0, newArray.length, ...newArray);
// console.log(float32Data.length)

4:13 - Property 'splice' does not exist on type 'Float32Array'. Did you mean 'slice'?


In [19]:
let a = [1,  2,  3,  4,  5]
let b = [10, 20, 30]

a.splice(0, b.length, ...b)

console.log(a) 

[ 10, 20, 30, 4, 5 ]


In [74]:


async function runModel(model, preprocessedData) {
  const start = new Date();
  try {
    const input = preprocessedData;
    const feeds = {float_input: input};
    // feeds[model.inputNames[0]] = preprocessedData;
    const outputData = await model.run(feeds);
    const end = new Date();
    const inferenceTime = (end.getTime() - start.getTime());
    const output = outputData[model.outputNames[0]];
    return [output, inferenceTime];
  } catch (e) {
    console.error(e);
    throw new Error();
  }
}

//The softmax transforms values to be between 0 and 1
function softmax(resultArray) {
  // Get the largest value in the array.
  const largestNumber = Math.max(...resultArray);
  // Apply exponential function to each result item subtracted by the largest number, use reduce to get the previous result number and the current number to sum all the exponentials results.
  const sumOfExp = resultArray.map((resultItem) => Math.exp(resultItem - largestNumber)).reduce((prevNumber, currentNumber) => prevNumber + currentNumber);
  //Normalizes the resultArray by dividing by the sum of all exponentials; this normalization ensures that the sum of the components of the output vector is 1.
  return resultArray.map((resultValue, index) => {
    return Math.exp(resultValue - largestNumber) / sumOfExp;
  });
}

const [res, time] =  await runModel(session, data);
var output = res.data;
var inferenceTime = time;
var results = softmax(Array.prototype.slice.call(output))

// var topResults = [];
// for (let i = 0; i < results.length; i++) {
//   if (results[i] > 0.3) {
//     topResults.push([classes[i] + ": " + results[i]]);
//   }
// }

console.log(inferenceTime);

TypeError: Only absolute URLs are supported
    at getNodeRequestOptions (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1305:9)
    at /home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1410:19
    at new Promise (<anonymous>)
    at fetch (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1407:9)
    at Object.createSessionHandler (/home/thuy/repo/deepsyence/node_modules/onnxruntime-web/dist/ort-web.node.js:6:159912)
    at Function.create (/home/thuy/repo/deepsyence/node_modules/onnxruntime-common/dist/ort-common.node.js:6:6607)
    at async evalmachine.<anonymous>:5:17
    at async Object.execute (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/executor.js:175:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/jupyter.js:219:18)


In [ ]:

async function runModel(model, preprocessedData) {
    const start = new Date();
    try {
      const input = new ort.Tensor(new Float32Array(preprocessedData), [1, 380]);
      const feeds = {float_input: input};
      // feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

In [ ]:
//The softmax transforms values to be between 0 and 1
function softmax(resultArray) {
  // Get the largest value in the array.
  const largestNumber = Math.max(...resultArray);
  // Apply exponential function to each result item subtracted by the largest number, use reduce to get the previous result number and the current number to sum all the exponentials results.
  const sumOfExp = resultArray.map((resultItem) => Math.exp(resultItem - largestNumber)).reduce((prevNumber, currentNumber) => prevNumber + currentNumber);
  //Normalizes the resultArray by dividing by the sum of all exponentials; this normalization ensures that the sum of the components of the output vector is 1.
  return resultArray.map((resultValue, index) => {
    return Math.exp(resultValue - largestNumber) / sumOfExp;
  });
}

In [75]:
const [res, time] =  await runModel(session, data);
var output = res.data;
var inferenceTime = time;
var results = softmax(Array.prototype.slice.call(output))

1:28 - Cannot find name 'runModel'.
1:37 - Cannot find name 'session'.
4:15 - Cannot find name 'softmax'.


In [ ]:
var topResults = [];
for (let i = 0; i < results.length; i++) {
  if (results[i] > 0.3) {
    topResults.push([classes[i] + ": " + results[i]]);
  }
}

console.log(topResults);